In [18]:
class data:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
args = data()
args.filename = "soya_dataset_year.csv"
args.stemmer = "lemma"
args.dictionary = "technology"

In [19]:
# -*- coding: utf-8 -*-
import numpy as np
import lda
import lda.datasets
import csv
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from gensim import corpora, models

from stemming.porter2 import stem
from nltk.stem import *
import unicodecsv
import re
import pyLDAvis.gensim
import gensim

_digits = re.compile('\d')
def contains_digits(d):
    return bool(_digits.search(d))

import enchant
d = enchant.Dict("en_US")
# Or using the /usr/share/dict/british-english word list
with open(args.dictionary + "-english") as word_file:
  english_words = set(word.strip().lower() for word in word_file)
  print(english_words)
  def is_english_word(word):
    return word.lower() in english_words

def process_tokens(tokens,stemmer):
  tokens = [i for i in tokens if not i in en_stop and not contains_digits(i) and is_english_word(i)]
  if stemmer == 'porter':
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(i) for i in tokens]
  elif stemmer == 'porter2':
    tokens = [stem(i) for i in tokens]
  elif stemmer == 'lemma':
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(i) for i in tokens]
  return tokens


if args.filename == "sample":
  X = lda.datasets.load_reuters()
  dictionary = lda.datasets.load_reuters_vocab()
  titles = lda.datasets.load_reuters_titles()
else:
  f = open(args.filename)
  reader = unicodecsv.reader(f, encoding='utf-8')
  # csv_length = sum(1 for row in reader)
  # f.seek(0) #reset reader position
  identifiers = reader.next()
  contents_idx = identifiers.index("contents")
  title_idx = identifiers.index("title")

  contents = [ row[contents_idx] for row in reader if row[contents_idx] ]

  f.seek(0)
  reader.next()
  titles = [ row[title_idx] for row in reader if row[contents_idx] ]



  texts = list()
  tokenizer = RegexpTokenizer(r'\w+')
  en_stop = get_stop_words('en')
  for idx,i in enumerate(contents):
#     if not idx % 10:
#       print "INFO: Tokenizing articles <{}> ".format(idx)
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    texts.append(process_tokens(tokens, args.stemmer))
    # print idx
    # add tokens to list

  print "[DEBUG] Length of Texts : {}".format(len(texts))
  dictionary = corpora.Dictionary(texts)
  corpus = [dictionary.doc2bow(text) for text in texts]


set(['', 'help desk desk', 'emerging', 'protocol', 'rom', 'gradual', 'hypertext', 'random', 'web page master', 'focus', 'jpeg', 'video conference conference', 'log off off', 'bandwidth', 'radio', 'innovative', 'screen', 'query', 'disk', 'speed', 'resource', 'video', 'layout', 'knowledge', 'copyright', 'personal', 'banner', 'revenue', 'megabyte', 'home page page', 'desktop', 'blog', 'source', 'topic', 'cad', 'integrate', 'input', 'retrieval', 'administrative', 'laptop', 'user name id', 'ranking', 'fax', 'crash', 'advantage', 'format', 'evaluate', 'photography', 'internet', 'overall', 'execute', 'wave', 'grid', 'usual', 'web master page', 'television', 'misuse', 'laser', 'domain name name', 'password', 'exact', 'disadvantage', 'daemon', 'basics', 'geek squad squad', 'edit', 'media specialist', 'journalism', 'upload', 'informative', 'benefit', 'history', 'journalist', 'activity', 'debug', 'output', 'security', 'trackball', 'media specialist specialist', 'user id id', 'worldwide', 'web mas

In [20]:
pyLDAvis.enable_notebook()
model_filename = args.filename.split('.')[0] + '.model'
try:
  ldamodel = models.LdaModel.load(model_filename)
except IOError:
  ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)
  ldamodel.save(model_filename)
    
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
8      20.825636        1       1  0.215833 -0.035621
0      18.251737        1       2  0.238224 -0.056739
7      12.701402        1       3  0.051032 -0.091526
3       9.652769        1       4 -0.223812 -0.290183
9       7.519502        1       5 -0.094998  0.046508
2       7.247652        1       6 -0.136407  0.003109
5       6.948310        1       7  0.164025  0.044816
6       6.934467        1       8  0.006257  0.104903
4       5.048536        1       9 -0.211298  0.196616
1       4.869990        1      10 -0.008856  0.078117, topic_info=    Category         Freq         Term  Total  loglift  logprob
194  Default   610.000000         data    610  30.0000  30.0000
135  Default  1592.000000       camera   1592  29.0000  29.0000
191  Default   397.000000       update    397  28.0000  28.0000
89   Default  2000.000000          new   2000  27.0000  27.0000
88   Default   374.000000     internet    374  26.0000  26.0000
11   Default   289.000000     keyboard    289  25.0000  25.0000
69   Default   398.000000        video    398  24.0000  24.0000
162  Default  1240.000000      display   1240  23.0000  23.0000
148  Default   269.000000         ipad    269  22.0000  22.0000
114  Default  1020.000000    processor   1020  21.0000  21.0000
24   Default   345.000000       window    345  20.0000  20.0000
137  Default   304.000000        store    304  19.0000  19.0000
96   Default   828.000000       screen    828  18.0000  18.0000
115  Default   202.000000     software    202  17.0000  17.0000
125  Default   460.000000       market    460  16.0000  16.0000
207  Default   410.000000   resolution    410  15.0000  15.0000
167  Default   145.000000        share    145  14.0000  14.0000
23   Default  1183.000000       source   1183  13.0000  13.0000
79   Default   368.000000     capacity    368  12.0000  12.0000
143  Default   149.000000     wireless    149  11.0000  11.0000
85   Default   172.000000       access    172  10.0000  10.0000
13   Default   138.000000        speed    138   9.0000   9.0000
140  Default   108.000000       medium    108   8.0000   8.0000
72   Default   145.000000     download    145   7.0000   7.0000
161  Default   461.000000       design    461   6.0000   6.0000
66   Default    76.000000         zoom     76   5.0000   5.0000
67   Default   145.000000         rate    145   4.0000   4.0000
25   Default    72.000000      program     72   3.0000   3.0000
173  Default   196.000000   technology    196   2.0000   2.0000
150  Default   101.000000      account    101   1.0000   1.0000
..       ...          ...          ...    ...      ...      ...
68   Topic10    12.496382     computer     37   1.9803  -4.3770
14   Topic10     5.683107       layout     18   1.9413  -5.1366
64   Topic10     2.622458   innovative     10   1.7792  -5.8865
26   Topic10     0.815353    integrate      4   1.6807  -6.9012
33   Topic10    12.213003    advantage     53   1.5724  -4.4255
4    Topic10     0.921313      gradual      5   1.5203  -6.8385
95   Topic10    32.155025         word    157   1.4456  -3.4663
34   Topic10     4.566603       format     23   1.4315  -5.4013
201  Topic10     1.958533   popularity     11   1.3792  -6.1912
18   Topic10     2.068472        exact     12   1.2941  -6.1893
173  Topic10    34.771179   technology    196   1.2864  -3.4038
198  Topic10     6.214782       remote     37   1.2532  -5.1041
177  Topic10     6.713488      variety     45   1.1293  -5.0323
80   Topic10     9.228756    broadband     84   0.8245  -4.7129
82   Topic10     7.473704       review     71   0.8061  -4.8995
29   Topic10    12.654112        pixel    136   0.5998  -4.4558
89   Topic10    67.662135          new   2000  -0.3760  -2.7434
96   Topic10    38.691324       screen    828  -0.0873  -3.3365
132  Topic10    22.491332      support    390   0.1669  -3.8352
188  Topic10    18.031395      product    279